# Display Helper
Clean display utilities for status and progress.
Run this with: `%run /home/jovyan/work/system/helpers/04_Display.ipynb`

In [ ]:
# Load dependencies
if 'tabulate' not in globals():
    from tabulate import tabulate
    import pandas as pd
    from datetime import datetime

In [ ]:
def print_header(title: str, width: int = 60):
    """Print a formatted header."""
    print("=" * width)
    print(f" {title}")
    print("=" * width)

def print_section(title: str, width: int = 60):
    """Print a formatted section header."""
    print("-" * width)
    print(f" {title}")
    print("-" * width)

def format_timestamp(ts):
    """Format timestamp for display."""
    if pd.isna(ts) or ts is None:
        return "-"
    if isinstance(ts, str):
        ts = pd.to_datetime(ts)
    return ts.strftime("%Y-%m-%d %H:%M:%S")

def format_duration(seconds):
    """Format duration in seconds to human readable."""
    if pd.isna(seconds) or seconds is None:
        return "-"
    
    seconds = int(seconds)
    if seconds < 60:
        return f"{seconds}s"
    elif seconds < 3600:
        mins = seconds // 60
        secs = seconds % 60
        return f"{mins}m {secs}s"
    else:
        hours = seconds // 3600
        mins = (seconds % 3600) // 60
        return f"{hours}h {mins}m"

def display_dataframe(df: pd.DataFrame, title: str = None, max_rows: int = None):
    """Display a DataFrame with nice formatting."""
    if title:
        print_section(title)
    
    if df.empty:
        print("No data available")
        return
    
    # Limit rows if specified
    if max_rows and len(df) > max_rows:
        df = df.head(max_rows)
        print(f"Showing first {max_rows} rows")
    
    # Format timestamps
    for col in df.columns:
        if 'ts' in col.lower() or 'time' in col.lower() or '_at' in col.lower():
            df[col] = df[col].apply(format_timestamp)
        elif 'duration' in col.lower() or 'seconds' in col.lower():
            df[col] = df[col].apply(format_duration)
    
    print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))

def display_status_table(data: list, headers: list = None):
    """Display a status table."""
    if not headers:
        headers = ['Property', 'Value']
    print(tabulate(data, headers=headers, tablefmt='grid'))

def display_progress_bar(current: int, total: int, width: int = 40, title: str = ""):
    """Display a text-based progress bar."""
    if total == 0:
        percent = 0
    else:
        percent = (current / total) * 100
    
    filled = int(width * current / max(total, 1))
    bar = '█' * filled + '░' * (width - filled)
    
    if title:
        print(f"{title}:")
    print(f"[{bar}] {percent:.1f}% ({current}/{total})")

def display_step_status(status: str) -> str:
    """Get status icon and text."""
    status_map = {
        'completed': '✅ Completed',
        'running': '⏳ Running',
        'failed': '❌ Failed',
        'skipped': '⏭️ Skipped',
        'pending': '⭕ Pending',
        None: '⭕ Not Started'
    }
    return status_map.get(status, f'❓ {status}')

def display_cycle_summary(cycle_info: dict):
    """Display cycle summary information."""
    print_header(f"Cycle: {cycle_info.get('cycle_name', 'Unknown')}")
    
    data = [
        ['Status', cycle_info.get('status', '-')],
        ['Created', format_timestamp(cycle_info.get('created_ts'))],
        ['Created By', cycle_info.get('created_by', '-')],
        ['Stages', cycle_info.get('stages', 0)],
        ['Steps', cycle_info.get('steps', 0)]
    ]
    
    if cycle_info.get('status') == 'archived':
        data.append(['Archived', format_timestamp(cycle_info.get('archived_ts'))])
    
    display_status_table(data)

def display_step_progress(progress_df: pd.DataFrame):
    """Display step progress in a clean format."""
    if progress_df.empty:
        print("No steps defined")
        return
    
    current_stage = None
    
    for _, row in progress_df.iterrows():
        # Print stage header if changed
        if row['stage_num'] != current_stage:
            current_stage = row['stage_num']
            print(f"\n📁 Stage {row['stage_num']}: {row['stage_name']}")
            print("  " + "-" * 50)
        
        # Print step status
        status_text = display_step_status(row.get('last_status'))
        idempotent = "🔄" if row.get('is_idempotent') else "🔒"
        
        print(f"  {idempotent} Step {row['step_num']}: {row['step_name']}")
        print(f"      Status: {status_text}")
        
        if row.get('last_run'):
            print(f"      Last Run: {format_timestamp(row['last_run'])}")

def clear_output():
    """Clear notebook output."""
    from IPython.display import clear_output as clear
    clear(wait=True)

print("✅ Display utilities loaded")